XGBoost
===

In [1]:
import xgboost as xgb
from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np
import scipy as sc
from sklearn import preprocessing
from sklearn.grid_search import GridSearchCV

Algorithms
---

In [2]:
# Parsing

#train_set = pd.read_csv('https://media.githubusercontent.com/media/riccamastellone/BIP/master/dataset/output/dataset_polimi_extended_with_province.csv?token=AJr57772_45EhdrKvwKTKj088rhOjaNwks5XcTOFwA%3D%3D',delimiter=',')
train_set = pd.read_csv('https://raw.githubusercontent.com/riccamastellone/BIP/master/dataset/output/dataset_polimi_extended_no2014.csv?token=AJr576cQljkvIkao9t78T4EZwajLHZvkks5Xe29WwA%3D%3D',delimiter=',')
#test_set = pd.read_csv('https://raw.githubusercontent.com/riccamastellone/BIP/master/dataset/output/test_with_features.csv?token=AJr572V5PSQQk_et0pEphHlyIPeQGxChks5XcT0gwA%3D%3D',delimiter=',')
test_set = pd.read_csv('https://raw.githubusercontent.com/riccamastellone/BIP/master/dataset/output/test_with_features.csv?token=AJr570gdvqYtMTESBVV4kcyqmrzIDar5ks5Xe2_AwA%3D%3D',delimiter=',')
#test_set = pd.read_csv("https://raw.githubusercontent.com/riccamastellone/BIP/master/dataset/output/test_with_features_with_provincie.csv?token=AJr577ZlQvRoERZYYis2-iIT_trl1r7Jks5Xel_4wA%3D%3D",delimiter=',')
pd.options.mode.chained_assignment = None
#Id_ts = test_['ID']
#test_set.drop('ID',axis=1,inplace=True)

In [3]:
train_set.head()

,zona,area,sottoarea,categoria,timestamp,giorno_della_settimana,giorno_del_mese,giorno_dell_anno,settimana_dell_anno,mese,...,regione_PUG,regione_SAR,regione_SIC,regione_TAA,regione_TOS,regione_UMB,regione_VEN,zona_italia_Centro,zona_italia_Nord,zona_italia_Sud
0,1,1,5,1,1420070400,3,1,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,1,5,1,1420156800,4,2,2,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1,1,5,1,1420243200,5,3,3,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,1,5,1,1420329600,6,4,4,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,1,5,1,1420416000,0,5,5,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
test_set.head()

,zona,area,sottoarea,categoria,timestamp,giorno_della_settimana,giorno_del_mese,giorno_dell_anno,settimana_dell_anno,mese,...,regione_PUG,regione_SAR,regione_SIC,regione_TAA,regione_TOS,regione_UMB,regione_VEN,zona_italia_Centro,zona_italia_Nord,zona_italia_Sud
0,10,26,67,1,1463702400,4,20,141,20,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,10,26,67,2,1463702400,4,20,141,20,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10,26,67,1,1463788800,5,21,142,20,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,10,26,67,2,1463788800,5,21,142,20,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,10,26,67,1,1463875200,6,22,143,21,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [5]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145440 entries, 0 to 145439
Data columns (total 44 columns):
zona                      145440 non-null int64
area                      145440 non-null int64
sottoarea                 145440 non-null int64
categoria                 145440 non-null int64
timestamp                 145440 non-null int64
giorno_della_settimana    145440 non-null int64
giorno_del_mese           145440 non-null int64
giorno_dell_anno          145440 non-null int64
settimana_dell_anno       145440 non-null int64
mese                      145440 non-null int64
anno                      145440 non-null int64
festivo                   145440 non-null int64
vendite                   145440 non-null int64
isola                     145440 non-null int64
abitanti                  145440 non-null int64
min_distanza              145440 non-null int64
densita                   145440 non-null int64
indice_istruzione         143420 non-null float64
reddito_medio          

In [6]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2880 entries, 0 to 2879
Data columns (total 43 columns):
zona                      2880 non-null int64
area                      2880 non-null int64
sottoarea                 2880 non-null int64
categoria                 2880 non-null int64
timestamp                 2880 non-null int64
giorno_della_settimana    2880 non-null int64
giorno_del_mese           2880 non-null int64
giorno_dell_anno          2880 non-null int64
settimana_dell_anno       2880 non-null int64
mese                      2880 non-null int64
anno                      2880 non-null int64
festivo                   2880 non-null int64
isola                     2880 non-null int64
abitanti                  2880 non-null int64
min_distanza              2880 non-null int64
densita                   2880 non-null int64
indice_istruzione         2840 non-null float64
reddito_medio             2880 non-null int64
gini_index                2880 non-null float64
forza_lavoro   

In [7]:
y_train = train_set.pop('vendite')

In [32]:
cv_params = { 'max_depth':[1,10,100],'min_child_weight':[1,10,100] }
ind_params = { 'subsample':0.9,'learning_rate':0.01,'n_estimators': 500, 'seed':0, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', "nthread":8}

optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params),
                            cv_params,
                            scoring = 'accuracy',
                            cv = 3, 
                            n_jobs = -1)

In [33]:
optimized_GBM.fit(train_set,y_train)

KeyboardInterrupt: 

In [12]:
optimized_GBM.grid_scores_

[mean: 0.25241, std: 0.01767, params: {'max_depth': 10, 'min_child_weight': 10},
 mean: 0.25140, std: 0.01398, params: {'max_depth': 10, 'min_child_weight': 70},
 mean: 0.25334, std: 0.01664, params: {'max_depth': 70, 'min_child_weight': 10},
 mean: 0.25571, std: 0.01535, params: {'max_depth': 70, 'min_child_weight': 70}]

In [27]:
optimized_GBM.grid_scores_

[mean: 0.23804, std: 0.01847, params: {'max_depth': 20, 'min_child_weight': 15},
 mean: 0.23888, std: 0.01809, params: {'max_depth': 20, 'min_child_weight': 23},
 mean: 0.23982, std: 0.01825, params: {'max_depth': 20, 'min_child_weight': 30},
 mean: 0.24055, std: 0.01800, params: {'max_depth': 20, 'min_child_weight': 36},
 mean: 0.23947, std: 0.01852, params: {'max_depth': 27, 'min_child_weight': 15},
 mean: 0.23969, std: 0.01951, params: {'max_depth': 27, 'min_child_weight': 23},
 mean: 0.23954, std: 0.01913, params: {'max_depth': 27, 'min_child_weight': 30},
 mean: 0.24077, std: 0.01718, params: {'max_depth': 27, 'min_child_weight': 36},
 mean: 0.23880, std: 0.01710, params: {'max_depth': 35, 'min_child_weight': 15},
 mean: 0.23902, std: 0.01720, params: {'max_depth': 35, 'min_child_weight': 23},
 mean: 0.24024, std: 0.01795, params: {'max_depth': 35, 'min_child_weight': 30},
 mean: 0.24097, std: 0.01906, params: {'max_depth': 35, 'min_child_weight': 36},
 mean: 0.23973, std: 0.01706

In [132]:
optimized_GBM.grid_scores_

[mean: 0.23313, std: 0.03200, params: {'max_depth': 7, 'min_child_weight': 5},
 mean: 0.23312, std: 0.03298, params: {'max_depth': 7, 'min_child_weight': 7},
 mean: 0.23236, std: 0.03461, params: {'max_depth': 7, 'min_child_weight': 9},
 mean: 0.23517, std: 0.03218, params: {'max_depth': 9, 'min_child_weight': 5},
 mean: 0.23539, std: 0.03347, params: {'max_depth': 9, 'min_child_weight': 7},
 mean: 0.23425, std: 0.03682, params: {'max_depth': 9, 'min_child_weight': 9},
 mean: 0.23394, std: 0.03378, params: {'max_depth': 11, 'min_child_weight': 5},
 mean: 0.23414, std: 0.03398, params: {'max_depth': 11, 'min_child_weight': 7},
 mean: 0.23394, std: 0.03382, params: {'max_depth': 11, 'min_child_weight': 9}]

In [ ]:
def custom_evaluation(preds, dtrain):
    labels = dtrain.get_label()
    # return a pair metric_name, result
    # since preds are margin(before logistic transformation, cutoff at 0)
    return 'error', float(sum(labels != (preds > 0.0))) / len(labels)

In [8]:
xgdmat = xgb.DMatrix(train_set, y_train, missing=np.nan)

In [14]:
our_params = {'eta': 0.1, 'seed':0,'learning_rate':0.01, 'subsample': 0.9, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':70, 'min_child_weight':70, "nthread":8} 
# Grid Search CV optimized settings

cv_xgb = xgb.cv(params = our_params, dtrain = xgdmat, num_boost_round = 3000, nfold = 5,
                metrics = ['error'], 
                early_stopping_rounds = 100)

Will train until cv error hasn't decreased in 100 rounds.
Stopping. Best iteration: 562


In [9]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':10, 'min_child_weight':10} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 562)

In [10]:
testdmat = xgb.DMatrix(test_set, missing=np.nan)

In [11]:
y_pred = final_gb.predict(testdmat) # Predict using our testdmat
y_pred

array([ 2.71868658,  0.97682607, -0.48832482, ...,  1.71807432,
        0.43935814,  0.26081297], dtype=float32)

In [17]:
y_pred[:10]

array([ 2.71868658,  0.97682607, -0.48832482,  0.43961689, -1.20718062,
       -0.38496959,  3.04898787,  1.1797297 ,  2.63060451,  1.46600938], dtype=float32)

In [13]:
final_df = test_set

In [18]:
final_df['prediction'] = y_pred

In [19]:
final_df.head()

,zona,area,sottoarea,categoria,timestamp,giorno_della_settimana,giorno_del_mese,giorno_dell_anno,settimana_dell_anno,mese,...,regione_SAR,regione_SIC,regione_TAA,regione_TOS,regione_UMB,regione_VEN,zona_italia_Centro,zona_italia_Nord,zona_italia_Sud,prediction
0,10,26,67,1,1463702400,4,20,141,20,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.718687
1,10,26,67,2,1463702400,4,20,141,20,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.976826
2,10,26,67,1,1463788800,5,21,142,20,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.488325
3,10,26,67,2,1463788800,5,21,142,20,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.439617
4,10,26,67,1,1463875200,6,22,143,21,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.207181


In [21]:
final_df.to_csv("xgboost_depth10_child10.csv", sep=',')